In [ ]:
import altair as alt
import pandas as pd

alt.data_transformers.disable_max_rows()

df = pd.read_csv('EU-UK-Weather-Data.csv')
df['date'] = pd.to_datetime(df['date'])
df.columns

In [ ]:
WIDTH, HEIGHT = 800, 600

time_brush = alt.selection_interval()
city_selection = alt.selection_point(fields=['location_name'], bind='legend')
metric_dropdown = alt.binding_select(
    options=['temperature_celsius', 'humidity'],
    labels=['Temperature °C', 'Humidity %'],
    name='Metric '
)
metric_selection = alt.param(value='temperature_celsius', bind=metric_dropdown)
search_box = alt.param(
    value='',
    bind=alt.binding(input='search', placeholder='City', name='Search ')
)
search_matches = alt.expr.test(alt.expr.regexp(search_box, 'i'), alt.datum.location_name)
condition_filter = alt.selection_point(fields=['condition'])
panning_selection = alt.selection_interval(encodings=['x'], bind='scales')

heatmap = alt.Chart(df).mark_rect().encode(
    x=alt.X('week:O', title='Week #'),
    y=alt.Y('location_name:N', title='City'),
    color=alt.condition(
        time_brush,
        alt.Color('metric:Q',
                  scale=alt.Scale(scheme='viridis'),
                  legend=alt.Legend(orient='right', title='Metric')),
        alt.value('lightgray')),
    tooltip=[alt.Tooltip('location_name', title='City'),
             alt.Tooltip('date', title='Date'),
             alt.Tooltip('temperature_celsius', title='Temperature (°C)'),
             alt.Tooltip('humidity', title='Humidity (%)')],
).transform_filter(
    search_matches,
).transform_filter(
    city_selection
).transform_filter(
    panning_selection
).transform_calculate(
    metric=f'datum[{metric_selection.name}]'
).add_params(
    time_brush,
    metric_selection,
    search_box,
).properties(
    title='Weather Metric Heatmap',
    width=0.6 * WIDTH,
    height=0.7 * HEIGHT,
)

pie_chart = alt.Chart(df).mark_arc(innerRadius=50).encode(
    theta=alt.Theta('count():Q', title='Frequency'),
    color=alt.Color('condition:N',
                    scale=alt.Scale(scheme='category10'),
                    legend=alt.Legend(orient='right', title='Condition')),
    tooltip=[alt.Tooltip('condition', title='Condition'),
             alt.Tooltip('count()', title='No. of Days')],
    opacity=alt.condition(condition_filter, alt.value(1.0), alt.value(0.2)),
).add_params(
    condition_filter
).transform_filter(
    search_matches
).transform_filter(
    time_brush
).transform_filter(
    city_selection
).properties(
    title='Weather Condition Piechart',
    width=0.3 * WIDTH,
    height=0.7 * HEIGHT,
).interactive()

barchart = alt.Chart(df).mark_bar().encode(
    x=alt.X('date', title='Date'),
    y=alt.Y('metric:Q', title='Metric'),
    # column='location_name:N',  # for a grouped bar chart
    opacity=alt.condition(city_selection, alt.value(1.0), alt.value(0.1)),
    color=alt.Color('location_name:N',
                    legend=alt.Legend(orient='right', title='City', columns=2)),
    tooltip=[alt.Tooltip('location_name', title='City'),
             alt.Tooltip('date', title='Date'),
             alt.Tooltip('week', title='Week #'),
             alt.Tooltip('temperature_celsius', title='Temperature (°C)'),
             alt.Tooltip('humidity', title='Humidity (%)'),
             alt.Tooltip('condition', title='Condition'),],
).add_params(
    panning_selection,
    city_selection
).transform_filter(
    search_matches
).transform_filter(
    time_brush
).transform_filter(
    condition_filter
).transform_calculate(
    metric=f'datum[{metric_selection.name}]'
).properties(
    title='Weather Metric Barchart',
    width=WIDTH,
    height=0.3 * HEIGHT,
)

systemC = alt.vconcat(
    barchart,
    heatmap | pie_chart,
).resolve_scale(
    color='independent',
).properties(
    title='System C'
)
systemC.save('systemC.html')
systemC